In [1]:
!apt-get install -y libgl1-mesa-glx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [18]:
!pip install ipywidgets

     |████████████████████████████████| 121 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 81.7 MB/s eta 0:00:01


In [71]:
!mount -o remount /dev/shm

mount: /dev/shm: permission denied.


In [24]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16445 sha256=32ee934eed2055a50ce87144266e744660d1296812b03e0aaafb5e9de44e331d
  Stored in directory: /opt/ml/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch


In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
from torchvision import transforms
import albumentations
import albumentations.pytorch
import os
from pathlib import Path
import cv2
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet

In [2]:
train_data = pd.read_csv('./input/data/train/train.csv')
eval_data = pd.read_csv('./input/data/eval/info.csv')
train_img_path = './input/data/train/images/'
eval_img_path = './input/data/eval/images/'

In [3]:
train_data

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [4]:
cpy_train_data = train_data.copy()

In [5]:
# class 판별 기준
# 1. mask 착용 여부 : image file name으로 판별
# 2. 성별 : female, male
# 3. 나이 : < 30, 30 <= x < 60, >= 60


# labeling
# filename, gender, age로 class를 특정해 return한다.
# filename : image file의 이름. image file의 이름이 image의 마스크 착용 여부를 기록하고 있다.
# gender : 성별, Male or Female
# age : 나이, 30세 미만, 30~60세, 60세 이상
def labeling(filename, gender, age):
    
    # classes : Mask상태, Gender, Age의 tuple을 key로 하고 class num을 반환
    classes = dict()
    cls = 0
    for category_filename in ['wear', 'incorrect', 'notWear']:
        for category_gender in ['male', 'female']:
            for category_age in ['under30', 'between30and60', 'over60']:
                classes[category_filename, category_gender, category_age] = cls
                cls = cls + 1
                
    # filename을 통해 Mask상태 지정
    mask = ('wear' if filename[:4] == 'mask' else 'notWear' if filename[:6] == 'normal' else 'incorrect')
    
    # gender는 이미 Male or Female
    
    # age를 통해 age 상태 지정
    age = ('under30' if age < 30 else 'between30and60' if 30 <= age < 60 else 'over60')    
    
    
    return classes[mask, gender, age]
########################################
########################################

X = []
y = []

for i in range(cpy_train_data.shape[0]):
    
    # 각 폴더에 해당하는 gender와 age를 지정하고,
    # 내부 image file의 이름을 image_list로 저장한다.
    folder_path = train_img_path + cpy_train_data.loc[i, 'path'] + '/'
    gender = cpy_train_data.loc[i, 'gender']
    age = cpy_train_data.loc[i, 'age']
    image_list = os.listdir(folder_path)
    
    # image file이 window 형식(앞에 ._ 안붙음)이고 image file이면 labeling 한다.
    # 폴더 내 파일이름, class를 각각 list로 저장한다. (X, y로 만들예정)
    
    for filename in image_list:
        if filename[0] != '.':
            X.append(folder_path + filename)
            y.append(labeling(filename, gender, age))
    

In [6]:
labeled_df = pd.DataFrame()

In [7]:
labeled_df['image'] = X
labeled_df['target'] = y

In [8]:
class MaskDataset(Dataset):
    def __init__(self, df, transform=None, train=True):
        super().__init__()
        self.df = df
        self.X = self.df.image
        self.y = self.df.target
        self.transform = transform
        self.train = train
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = cv2.imread(self.X[idx])
        X = cv2.cvtColor(X, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            X = self.transform(image=X)['image']
        if self.train:
            y = self.y[idx]
        
        return X.clone().detach(), torch.tensor(y)
        

In [9]:
train=labeled_df.sample(frac=0.8,random_state=200) #random state is a seed value
test=labeled_df.drop(train.index)

In [11]:
train.index = [i for i in range(len(train.index))]
test.index = [i for i in range(len(test.index))]

In [12]:
train_data = MaskDataset(train,
                   transform=albumentations.Compose([
                       albumentations.HorizontalFlip(),
                       albumentations.ToGray(),
                       albumentations.Rotate(limit=30),
                       albumentations.Normalize(),
                       albumentations.pytorch.ToTensorV2()]))

test_data = MaskDataset(test,
                   transform=albumentations.Compose([
                       albumentations.ToGray(),
                       albumentations.Normalize(),
                       albumentations.pytorch.ToTensorV2()]))


In [14]:
train_dl = DataLoader(train_data, batch_size = 32, drop_last = False, shuffle=True, num_workers=1)
test_dl = DataLoader(test_data, batch_size = 32, drop_last = False, shuffle=True, num_workers=1)

In [15]:
next(iter(test_dl))

[tensor([[[[ 0.9132,  0.9132,  0.9132,  ...,  0.7762,  0.6906,  0.6221],
           [ 0.9132,  0.9132,  0.9132,  ...,  0.7248,  0.6734,  0.6221],
           [ 0.9132,  0.9132,  0.9132,  ...,  0.6563,  0.6221,  0.6221],
           ...,
           [-0.4739, -0.4568, -0.4568,  ..., -0.9363, -0.8335, -0.7822],
           [-0.4739, -0.4739, -0.4568,  ..., -0.9363, -0.8335, -0.7822],
           [-0.4739, -0.4739, -0.4739,  ..., -0.9363, -0.8507, -0.7822]],
 
          [[ 1.0630,  1.0630,  1.0630,  ...,  0.8880,  0.8004,  0.7304],
           [ 1.0630,  1.0630,  1.0630,  ...,  0.8354,  0.7829,  0.7304],
           [ 1.0630,  1.0630,  1.0630,  ...,  0.7654,  0.7304,  0.7304],
           ...,
           [-0.3375, -0.3200, -0.3200,  ..., -0.8803, -0.8102, -0.7577],
           [-0.3375, -0.3375, -0.3200,  ..., -0.8803, -0.8102, -0.7577],
           [-0.3375, -0.3375, -0.3375,  ..., -0.8803, -0.8277, -0.7577]],
 
          [[ 1.1411,  1.1411,  1.1411,  ...,  0.8797,  0.7925,  0.7228],
           [ 

In [16]:
#model = torchvision.models.resnet18(pretrained=True, progress=False)
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=18)
model.train()

Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [17]:
model.fc = torch.nn.Linear(in_features=512, out_features=18, bias=True)

In [18]:
for param in model.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)
model.to(device)

LEARNING_RATE = 0.001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 40 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

cuda:0


In [20]:
torch.cuda.empty_cache()

In [21]:
### 학습 코드 시작


for epoch in range(NUM_EPOCH):
    train_loss = 0.
    train_acc = 0.
    test_loss = 0.
    test_acc = 0.
    
    train_data_num = 0
    test_data_num = 0
    for inputs, labels in tqdm(train_dl):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = loss_fn(outputs, labels)     
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0) # 한 Batch에서의 loss 값 저장
        train_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
        train_data_num += len(preds)
        
    for inputs, labels in tqdm(test_dl):
        with torch.no_grad():
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
        loss = loss_fn(outputs, labels)
        
        test_loss += loss.item() * inputs.size(0)
        test_acc += torch.sum(preds == labels.data)
        test_data_num += len(preds)
            
            
        
    epoch_loss = train_loss / train_data_num
    epoch_acc = train_acc / train_data_num
    val_loss = test_loss / test_data_num
    val_acc = test_acc / test_data_num

    print(f"현재 epoch-{epoch}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    print(f"현재 epoch-{epoch}의 val Loss : {val_loss:.3f}, val Accuracy : {val_acc:.3f}")
    print("")

  0%|          | 0/473 [00:00<?, ?it/s]

현재 epoch-0의 평균 Loss : 0.498, 평균 Accuracy : 0.841
현재 epoch-0의 val Loss : 0.267, val Accuracy : 0.911



  0%|          | 0/473 [00:00<?, ?it/s]

현재 epoch-1의 평균 Loss : 0.244, 평균 Accuracy : 0.917
현재 epoch-1의 val Loss : 0.187, val Accuracy : 0.943



  0%|          | 0/473 [00:00<?, ?it/s]

...

현재 epoch-4의 평균 Loss : 0.111, 평균 Accuracy : 0.962
현재 epoch-4의 val Loss : 0.122, val Accuracy : 0.962



  0%|          | 0/473 [00:00<?, ?it/s]

현재 epoch-5의 평균 Loss : 0.088, 평균 Accuracy : 0.969
현재 epoch-5의 val Loss : 0.108, val Accuracy : 0.967



  1%|          | 3/473 [00:02<07:16,  1.08it/s]


KeyboardInterrupt: 

In [21]:
torch.cuda.empty_cache()

In [23]:
torch.save(model.state_dict(), os.path.join('/opt/ml/input/data', "model.pt"))

## Inference

In [22]:
test_dir = '/opt/ml/input/data/eval'
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')





#################################
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)
#################################



# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.ToTensor(),
])

###########################################
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')

model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())

submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 12600/12600 [10:48<00:00, 19.44it/s]

test inference is done!
